In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from metrics import *

import os
import pandas as pd
import numpy as np
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [ ]:
MODEL_NAME = 'cnn'
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

In [ ]:
# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

### load trained model

In [ ]:
# All comments must be truncated or padded to be the same length.
MAX_SEQUENCE_LENGTH = 250
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
model = load_model('./models/baseline_cnn.h5')
with open('./models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
test_df = pd.read_csv('./Data/test_private_expanded.csv')
try:
    test_df.rename(columns={'toxicity': 'target'}, inplace=True)
except:
    pass
test_df = convert_dataframe_to_bool(test_df)

In [ ]:
# test_df[MODEL_NAME] = model.predict(pad_text(test_df[TEXT_COLUMN], tokenizer))[:, 1]

In [ ]:
test_df.to_csv(f'./submissions/{MODEL_NAME}_submission.csv')